In [4]:
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd

url = "https://kpub.knia.or.kr/carInsuranceDisc/specialTerms/specialTerms.do"
res = requests.get(url, verify = False)
db_html = res.text

soup = BeautifulSoup(db_html, 'lxml')
table_div = soup.find_all('table')[1]
thead_rows = table_div.find("thead").find_all("tr")
headers = []

for th in thead_rows[0].find_all("th"):
    text = th.get_text(strip=True)
    colspan = th.get("colspan")
    if colspan:
        headers.extend([text] * int(colspan))
    else:
        headers.append(text)

# 두 번째 줄 헤더 반영
sub_headers = [th.get_text(strip=True) for th in thead_rows[1].find_all("th")]

# 헤더 병합
print(headers)
print(sub_headers)
headers[5] = sub_headers[0]
headers[-1] = sub_headers[1]
headers


# 3. 데이터 추출
data_rows = []
for row in table_div.find("tbody").find_all("tr"):
    cols = [td.get_text(strip=True) for td in row.find_all(["td", "th"])]
    data_rows.append(cols)

# 4. DataFrame으로 만들기
df = pd.DataFrame(data_rows, columns=headers)


# 5. 공백/특수문자 처리
def clean_text(text):
    if pd.isna(text):
        return ""
    return (text
            .replace('\u00A0', ' ')  # non-breaking space → 일반 공백
            .replace('\u200B', '')   # zero-width space → 삭제
            .replace('\t', '')
            .replace('\n', '')
            .replace('\r', '')
            .strip()
    )

df["가입조건"] = df["가입조건"].astype(str).apply(clean_text)

# 5. CSV로 출력
df.to_csv("car_ins.csv", index=False, encoding='utf-8-sig')
df

C:\Users\Playdata\AppData\Local\miniconda3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kpub.knia.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


['구분', '회사명', '특약명', '가입조건', '판매채널', '할인율', '할인율']
['적용담보(자동차상해 및담보별 확장특약 포함기준)', '할인율(%)ascdesc']


,구분,회사명,특약명,가입조건,판매채널,적용담보(자동차상해 및담보별 확장특약 포함기준),할인율(%)ascdesc
0,자녀할인,메리츠화재,자녀(만7세이하) 할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,대면/TM/CM/PM,대인1/대인2/대물/자손/무보험/자차,7
1,자녀할인,한화손보,자녀할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,"대면, TM","대인1, 대인2, 대물, 자손, 무보험차상해, 자차",24.4
2,자녀할인,한화손보,자녀할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,"대면, TM","대인1, 대인2, 대물, 자손, 무보험차상해, 자차",20.1
3,자녀할인,한화손보,자녀할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,"대면, TM","대인1, 대인2, 대물, 자손, 무보험차상해, 자차",20.6
4,자녀할인,한화손보,자녀할인 특별약관,▶ 다음의 조건 모두 해당하는 경우에 한하여 가입 가능① 자녀(태아) 연령조건 : ...,"대면, TM","대인1, 대인2, 대물, 자손, 무보험차상해, 자차",16.1
...,...,...,...,...,...,...,...
426,서민(나눔)우대할인,DB손보,프로미하트(서민우대 보험료 할인) 특별약관,(상동),대면,대인1/대인2/대물/\n자손/무보험/자차,8
427,서민(나눔)우대할인,DB손보,프로미하트나눔(서민우대 보험료 할인) 특별약관,(상동),TM/CM/플랫폼,대인2/대물/\n자손/무보험/자차,5.4
428,서민(나눔)우대할인,DB손보,프로미하트나눔(서민우대 보험료 할인) 특별약관,(상동),TM/CM/플랫폼,대인1,3.4
429,서민(나눔)우대할인,AXA손보,나눔(서민우대) 특별약관,(상동),"TM, CM","대인1, 대인2, 대물, 자손, 무보험차상해, 자차",3.6


In [5]:
text = "삼성화재 가입조건"
print([ord(c) for c in text])
# 일반 공백: 32

text2 = "삼성화재 가입조건"  # 여기 중간에 \u00A0
print([ord(c) for c in text2])
# 중간에 160 (non-breaking space) 나옴


[49340, 49457, 54868, 51116, 32, 44032, 51077, 51312, 44148]
[49340, 49457, 54868, 51116, 160, 44032, 51077, 51312, 44148]


In [32]:
# CSV 파일 불러오기
df = pd.read_csv("car_ins.csv", encoding="utf-8")

# 특정 행의 가입조건 확인 (예: 0번 행)
text = df.loc[414, "가입조건"]  # 또는 인덱스 번호를 바꿔가며 확인해도 됨

# 글자 하나씩 유니코드(정수)로 출력
print("원본 텍스트:", repr(text))
print("문자별 유니코드 값:", [ord(c) for c in text])
print("문자별 출력:", [(c, ord(c)) for c in text])

원본 텍스트: '상동'
문자별 유니코드 값: [49345, 46041]
문자별 출력: [('상', 49345), ('동', 46041)]


In [37]:
import pandas as pd
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='SKN13_1st_4Team',
    password='1111',
    database='car_insurance',
    charset='utf8mb4'
)

df = pd.read_sql("SELECT * FROM car_ins", conn)
df.to_csv("car_ins_after_prepro.csv", index=False, encoding='utf-8-sig')
df = pd.read_sql("SELECT * FROM car_ins_com", conn)
df.to_csv("car_ins_com_after_prepro.csv", index=False, encoding='utf-8-sig') 

conn.close()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_2944\1673225574.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM car_ins", conn)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_2944\1673225574.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM car_ins_com", conn)
